In [12]:
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

In [13]:
class Stretch:
    def __init__(self, path: str):
        self.path = path
        with open('../data/test_mono.wav', 'rb') as f:
            self.wave, self.sr = sf.read(f)
    
    def stretch(self,
                rate: float=1,
                bins: int = 2048, 
                overlap: float = 0.75)->np.ndarray:
    
        fourier = signal.stft(self.wave,nperseg=bins,noverlap=bins*overlap)[2]
        voc = self.__phase_vocoder(fourier, rate=rate, hop_length=bins*overlap)
        y_stretch = signal.istft(voc, nperseg=bins,noverlap=bins*overlap)
        self.stretched_wave = y_stretch[1]
    
    def write(self, out_path: str):
        with open(out_path, 'wb') as f:
            sf.write(f, self.stretched_wave, self.sr)
        
        
    @staticmethod
    def __phase_vocoder(fourier: np.ndarray,
                        rate: float,
                        hop_length: int = None,
                        n_fft: int = None) -> np.ndarray:

        time_steps = np.arange(0, fourier.shape[-1], rate, dtype=np.float64)
        shape = list(fourier.shape)
        shape[-1] = len(time_steps)
        d_stretch = np.zeros_like(fourier, shape=shape)
        phi_advance = np.linspace(0, np.pi * hop_length, fourier.shape[-2])
        phase_acc = np.angle(fourier[:, 0])
        padding = [(0, 0) for _ in fourier.shape]
        padding[-1] = (0, 2)
        fourier = np.pad(fourier, padding, mode="constant")

        for t, step in enumerate(time_steps):
            columns = fourier[:, int(step) : int(step + 2)]
            alpha = np.mod(step, 1.0)
            mag = (1.0 - alpha) * np.abs(columns[:, 0]) + alpha * np.abs(columns[:, 1])
            d_stretch[:, t] = (np.cos(phase_acc) + 1j * np.sin(phase_acc))*mag
            dphase = np.angle(columns[:, 1]) - np.angle(columns[:, 0]) - phi_advance
            dphase = dphase - 2.0 * np.pi * np.round(dphase / (2.0 * np.pi))
            phase_acc += phi_advance + dphase

        return d_stretch

In [14]:
stretcher = Stretch('../data/test_mono.wav')
stretcher.stretch(rate=2)
stretcher.write('../data/test_mono_r2.wav')
stretcher.stretch(rate=0.5)
stretcher.write('../data/test_mono_r05.wav')